# PCA 逆变换与数据重构

## 概述

PCA 逆变换（Inverse Transform）是将降维后的数据映射回原始特征空间的过程。虽然降维是有损压缩，但通过逆变换可以近似重构原始数据。

## 数学原理

设原始数据为 $X \in \mathbb{R}^{m \times n}$，PCA 投影矩阵为 $W_d \in \mathbb{R}^{n \times d}$：

- **正向变换（降维）**：$X_{reduced} = X \cdot W_d$
- **逆变换（重构）**：$X_{reconstructed} = X_{reduced} \cdot W_d^\top$

## 重构误差

重构误差衡量了降维过程中的信息损失：

$$\text{MSE} = \frac{1}{m} \sum_{i=1}^{m} \|x_i - \hat{x}_i\|^2$$

## 本节内容

1. MNIST 数据集上的 PCA 压缩
2. 不同成分数下的重构效果
3. 重构图像可视化
4. 压缩率与质量的权衡

## 1. 环境准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error

# 设置随机种子
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 2. 数据加载

使用 MNIST 手写数字数据集：
- 70,000 张 28×28 灰度图像
- 展平为 784 维向量

In [ ]:
# 加载 MNIST 数据集（使用子集加速演示）
print("正在加载 MNIST 数据集...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X, y = mnist.data, mnist.target

# 使用子集进行快速演示
n_samples = 10000
indices = np.random.choice(len(X), n_samples, replace=False)
X_subset = X[indices].astype(np.float32)
y_subset = y[indices]

print(f"原始数据形状: {X.shape}")
print(f"子集数据形状: {X_subset.shape}")
print(f"每张图像维度: {X_subset.shape[1]} (28×28 像素)")

In [ ]:
# 归一化到 [0, 1]
X_normalized = X_subset / 255.0

print(f"像素值范围: [{X_normalized.min():.2f}, {X_normalized.max():.2f}]")

## 3. 不同成分数的 PCA 降维与重构

比较使用不同主成分数量时的重构效果。

In [ ]:
# 测试不同的主成分数量
n_components_list = [10, 30, 50, 100, 154, 300]

results = []

for n_comp in n_components_list:
    # 创建并训练 PCA
    pca = PCA(n_components=n_comp)
    X_reduced = pca.fit_transform(X_normalized)
    
    # 逆变换重构数据
    X_reconstructed = pca.inverse_transform(X_reduced)
    
    # 计算重构误差
    mse = mean_squared_error(X_normalized, X_reconstructed)
    explained_var = pca.explained_variance_ratio_.sum()
    compression_ratio = 784 / n_comp
    
    results.append({
        'n_components': n_comp,
        'mse': mse,
        'explained_variance': explained_var,
        'compression_ratio': compression_ratio,
        'pca': pca,
        'X_reconstructed': X_reconstructed
    })
    
    print(f"成分数: {n_comp:3d} | 解释方差: {explained_var:.4f} | "
          f"MSE: {mse:.6f} | 压缩比: {compression_ratio:.1f}x")

## 4. 重构效果可视化

展示同一张图像在不同压缩程度下的重构效果。

In [ ]:
# 选择几张图像进行可视化
n_display = 5
display_indices = np.random.choice(n_samples, n_display, replace=False)

fig, axes = plt.subplots(n_display, len(n_components_list) + 1, 
                         figsize=(16, 2.5 * n_display))

for row, idx in enumerate(display_indices):
    # 原始图像
    axes[row, 0].imshow(X_normalized[idx].reshape(28, 28), cmap='gray')
    axes[row, 0].set_title('Original' if row == 0 else '')
    axes[row, 0].axis('off')
    
    # 不同成分数的重构
    for col, result in enumerate(results):
        reconstructed = result['X_reconstructed'][idx].reshape(28, 28)
        axes[row, col + 1].imshow(reconstructed, cmap='gray')
        if row == 0:
            axes[row, col + 1].set_title(f"n={result['n_components']}\n"
                                         f"({result['explained_variance']*100:.1f}%)")
        axes[row, col + 1].axis('off')

plt.suptitle('PCA Reconstruction with Different Number of Components', 
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 5. 压缩效率分析

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

n_comps = [r['n_components'] for r in results]
mses = [r['mse'] for r in results]
explained_vars = [r['explained_variance'] for r in results]
compression_ratios = [r['compression_ratio'] for r in results]

# 重构误差 vs 成分数
axes[0].plot(n_comps, mses, 'o-', color='steelblue', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Components')
axes[0].set_ylabel('Mean Squared Error')
axes[0].set_title('Reconstruction Error vs Components')
axes[0].grid(True, alpha=0.3)

# 解释方差 vs 成分数
axes[1].plot(n_comps, explained_vars, 'o-', color='green', linewidth=2, markersize=8)
axes[1].axhline(y=0.95, color='red', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Explained Variance Ratio')
axes[1].set_title('Explained Variance vs Components')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MSE vs 压缩比
axes[2].plot(compression_ratios, mses, 'o-', color='orange', linewidth=2, markersize=8)
axes[2].set_xlabel('Compression Ratio')
axes[2].set_ylabel('Mean Squared Error')
axes[2].set_title('Reconstruction Error vs Compression')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 详细案例：154 成分的重构

154 个成分是常用的 MNIST 压缩设置，可保留约 95% 的方差。

In [ ]:
# 使用 154 成分进行详细分析
pca_154 = PCA(n_components=154)
X_reduced_154 = pca_154.fit_transform(X_normalized)
X_reconstructed_154 = pca_154.inverse_transform(X_reduced_154)

print(f"原始数据大小: {X_normalized.nbytes / 1024 / 1024:.2f} MB")
print(f"降维后数据大小: {X_reduced_154.nbytes / 1024 / 1024:.2f} MB")
print(f"压缩比: {X_normalized.nbytes / X_reduced_154.nbytes:.2f}x")
print(f"保留方差: {pca_154.explained_variance_ratio_.sum() * 100:.2f}%")
print(f"重构 MSE: {mean_squared_error(X_normalized, X_reconstructed_154):.6f}")

In [ ]:
# 每张图像的重构误差分布
per_image_mse = np.mean((X_normalized - X_reconstructed_154) ** 2, axis=1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 误差分布直方图
axes[0].hist(per_image_mse, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
axes[0].axvline(per_image_mse.mean(), color='red', linestyle='--', 
                label=f'Mean: {per_image_mse.mean():.4f}')
axes[0].set_xlabel('Mean Squared Error')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Per-Image Reconstruction Error')
axes[0].legend()

# 最好和最差重构示例
best_idx = np.argmin(per_image_mse)
worst_idx = np.argmax(per_image_mse)

# 创建对比子图
axes[1].set_visible(False)
gs = fig.add_gridspec(1, 4, left=0.55, right=0.98, wspace=0.1)
ax_best_orig = fig.add_subplot(gs[0, 0])
ax_best_recon = fig.add_subplot(gs[0, 1])
ax_worst_orig = fig.add_subplot(gs[0, 2])
ax_worst_recon = fig.add_subplot(gs[0, 3])

ax_best_orig.imshow(X_normalized[best_idx].reshape(28, 28), cmap='gray')
ax_best_orig.set_title('Best\nOriginal')
ax_best_orig.axis('off')

ax_best_recon.imshow(X_reconstructed_154[best_idx].reshape(28, 28), cmap='gray')
ax_best_recon.set_title(f'MSE:\n{per_image_mse[best_idx]:.4f}')
ax_best_recon.axis('off')

ax_worst_orig.imshow(X_normalized[worst_idx].reshape(28, 28), cmap='gray')
ax_worst_orig.set_title('Worst\nOriginal')
ax_worst_orig.axis('off')

ax_worst_recon.imshow(X_reconstructed_154[worst_idx].reshape(28, 28), cmap='gray')
ax_worst_recon.set_title(f'MSE:\n{per_image_mse[worst_idx]:.4f}')
ax_worst_recon.axis('off')

plt.tight_layout()
plt.show()

## 7. 主成分可视化

将前几个主成分可视化为图像，理解它们捕捉的特征。

In [ ]:
# 可视化前 20 个主成分
n_show = 20
fig, axes = plt.subplots(4, 5, figsize=(12, 10))

for i, ax in enumerate(axes.flat):
    if i < n_show:
        component = pca_154.components_[i].reshape(28, 28)
        ax.imshow(component, cmap='RdBu_r')
        ax.set_title(f'PC{i+1}\n({pca_154.explained_variance_ratio_[i]*100:.1f}%)')
    ax.axis('off')

plt.suptitle('Top 20 Principal Components (Eigenfaces)', fontsize=14)
plt.tight_layout()
plt.show()

## 8. 总结

### 关键发现

1. **压缩效率**：154 个成分（784维的约 20%）可保留 ~95% 的信息

2. **重构质量**：
   - 成分数越多，重构越精确
   - 边际效益递减：增加更多成分带来的改善越来越小

3. **实际应用考量**：
   - 压缩存储和传输
   - 去噪（低阶重构可去除高频噪声）
   - 异常检测（高重构误差可能表示异常）

### 逆变换的局限性

- 重构是近似的，无法完全恢复原始数据
- 丢失的信息对应被舍弃的主成分方向
- 对于高度非线性的数据结构，线性 PCA 的重构效果有限